This Notebook creates a new geodataframe that maps adjacent FIPS codes + geometries to each County. The Geodataframe is held in memory because 
you cannot write out a geodataframe that contains fields with lists. Could explode dataframe but currently it is not flattened. 

The csv file 'co_neighbor' was generated in ArcGIS using the Neighbor tool

In [2]:
def create_adj_gdf(county_df, csv):
    df_m = county_df.merge(csv, left_on='FIPSSTCO', right_on='nbr_FIPSSTCO')
    
    gb_fips = df_m.groupby('src_FIPSSTCO')['nbr_FIPSSTCO'].apply(list)
    gb_geom = df_m.groupby('src_FIPSSTCO')['geometry'].apply(list)

    d_geom = gb_geom.to_dict()
    d_fips = gb_fips.to_dict()

    county_df['n_geom'] = county_df['FIPSSTCO'].map(d_geom)
    county_df['n_fips'] = county_df['FIPSSTCO'].map(d_fips)

    return county_df

In [3]:
df = gpd.read_file('counties.shp')
df = df.to_crs(26913)
n = pd.read_csv('co_neighbor.csv', dtype={'src_FIPSSTCO': str, 'nbr_FIPSSTCO': str})

adj = create_adj_gdf(df, n)

In [4]:
adj

,FIPSSTCO,STATE,COUNTY,geometry,n_geom,n_fips
0,01001,Alabama,Autauga,"POLYGON ((2251335.152 3775489.906, 2252577.732...",[POLYGON ((2207670.0621667323 3809828.17613435...,"[01021, 01047, 01051, 01085, 01101]"
1,01003,Alabama,Baldwin,"POLYGON ((2149277.868 3593872.929, 2149433.148...",[POLYGON ((2148199.27171483 3674084.1674165037...,"[01025, 01053, 01097, 01099, 01129, 12033]"
2,01005,Alabama,Barbour,"POLYGON ((2377643.530 3733228.804, 2377550.346...",[POLYGON ((2313188.1908887876 3738827.55661973...,"[01011, 01045, 01067, 01109, 01113, 13061, 132..."
3,01007,Alabama,Bibb,"POLYGON ((2181247.545 3825792.133, 2181445.321...",[POLYGON ((2207670.0621667323 3809828.17613435...,"[01021, 01065, 01073, 01105, 01117, 01125]"
4,01009,Alabama,Blount,"POLYGON ((2215130.222 3950304.309, 2215579.468...",[POLYGON ((2157276.2936139875 3945684.87151775...,"[01043, 01055, 01073, 01095, 01115, 01127]"
...,...,...,...,...,...,...
3278,78030,Virgin Islands,Saint Thomas,"POLYGON ((5034511.025 2612671.687, 5038496.375...",[POLYGON ((5050322.081278175 2608442.056396387...,[78020]
3279,51770,Virginia,Roanoke (City),"POLYGON ((2735023.914 4439277.165, 2735063.839...",[POLYGON ((2730102.627182817 4435063.306752956...,"[51775, 51161]"
3280,51161,Virginia,Roanoke,"POLYGON ((2735276.766 4443165.071, 2735822.261...",[POLYGON ((2775008.3895753454 4483624.96829561...,"[51019, 51023, 51045, 51063, 51067, 51121, 517..."
3281,02066,Alaska,Copper River,"POLYGON ((-1439309.470 7665399.489, -1433785.5...",[POLYGON ((-1449398.092568043 7711074.92390418...,"[02170, 02240, 02282, 02063]"


In [5]:
adj_ls = adj.loc[adj['FIPSSTCO'] == select_gdb, 'n_fips'].to_list()
adj_ls = [fips for adj in adj_ls for fips in adj]
adj_ls

NameError: name 'select_gdb' is not defined